In [1]:
from google.colab import auth
from googleapiclient.discovery import build
from google.colab import drive
import os
import sys
auth.authenticate_user()
drive.mount('/content/drive')
nb_path = '/content/notebooks'

%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import nltk
from nltk.corpus import gutenberg
import os

# Descargar el corpus gutenberg si no está disponible
nltk.download('gutenberg')

# Configuración del tamaño objetivo (10 GB)
target_size_gb =10
target_size_bytes = target_size_gb * 1024**3  # Convertir GB a bytes

# Obtener el texto del corpus de Gutenberg
corpus = ' '.join(gutenberg.words('austen-emma.txt'))  # Usamos una obra de Jane Austen como ejemplo
text_length = len(corpus.encode('utf-8'))  # Longitud del texto en bytes

# Verificar que el corpus tiene datos
if text_length == 0:
    raise ValueError("El corpus está vacío o no se pudo cargar correctamente.")

# Función para generar un archivo de texto de tamaño objetivo
def generate_large_file(file_path, corpus, target_size_bytes):
    with open(file_path, 'wb') as f:
        # Duplicar el texto hasta alcanzar el tamaño objetivo
        written_size = 0
        corpus_bytes = corpus.encode('utf-8')
        while written_size < target_size_bytes:
            remaining_size = target_size_bytes - written_size
            chunk_size = min(len(corpus_bytes), remaining_size)
            f.write(corpus_bytes[:chunk_size])
            written_size += chunk_size

# Ruta del archivo de salida
file_name = "/content/drive/MyDrive/large_text_file.txt"

# Generar el archivo de texto
generate_large_file(file_name, corpus, target_size_bytes)

print(f"Archivo generado con éxito: {file_name}")


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Archivo generado con éxito: /content/drive/MyDrive/large_text_file.txt


In [6]:
%%writefile wordcounter.cu

#include <iostream>
#include <fstream>
#include <unordered_map>
#include <string>
#include <vector>
#include <cuda_runtime.h>


//para determinar si un carácter es parte de una palabra, ya sea una letra o un número.
__host__ __device__ bool is_word_char(char c) {
    return (c >= 'a' && c <= 'z') || (c >= 'A' && c <= 'Z') || (c >= '0' && c <= '9');
}


//Calcula el índice del carácter actual (idx) en el texto.
//Verifica si el carácter es parte de una palabra y si es el comienzo de una nueva palabra.
//Si es el inicio de una palabra, se incrementa un contador en word_counts usando atomicAdd para evitar condiciones de carrera entre hilos.
__global__ void count_words_kernel(char *text, int text_size, int *word_counts) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= text_size) return;

    if (is_word_char(text[idx]) && (idx == 0 || !is_word_char(text[idx - 1]))) {
        atomicAdd(&word_counts[idx], 1);
    }
}

void checkCudaError(cudaError_t err, const char *msg) {
    if (err != cudaSuccess) {
        std::cerr << msg << ": " << cudaGetErrorString(err) << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main() {
    std::ifstream file("large_text_file.txt", std::ios::binary | std::ios::ate);
    if (!file.is_open()) {
        std::cerr << "Error opening file!" << std::endl;
        return 1;
    }

    std::streamsize file_size = file.tellg();
    file.seekg(0, std::ios::beg);

    std::cout << "File size: " << file_size << " bytes" << std::endl;


    //Se define un tamaño de fragmento (chunk_size) de 1 MB para procesar el archivo en partes.
    //Se inicializan estructuras para almacenar el texto y los conteos de palabras (h_text y h_word_counts).

    const std::streamsize chunk_size = 500 * 1024 * 1024; // 10 MB
    std::size_t num_chunks = (file_size + chunk_size - 1) / chunk_size;

    std::unordered_map<std::string, int> total_word_count;

    char *h_text = new char[chunk_size];
    int *h_word_counts = new int[chunk_size];

    //Se calcula la posición inicial (start) y final (end) de cada fragmento.
    //Se lee el fragmento y se carga en h_text.
    for (std::size_t chunk_idx = 0; chunk_idx < num_chunks; ++chunk_idx) {
        std::streamsize start = chunk_idx * chunk_size;
        std::streamsize end = std::min(start + chunk_size, file_size);
        std::streamsize current_chunk_size = end - start;

        file.seekg(start, std::ios::beg);
        file.read(h_text, current_chunk_size);

        std::cout << "Processing chunk " << chunk_idx + 1 << " of " << num_chunks << ", size: " << current_chunk_size << " bytes" << std::endl;

        char *d_text;
        int *d_word_counts;
        //Se reserva memoria en el GPU para d_text y d_word_counts.
        cudaMalloc(&d_text, current_chunk_size * sizeof(char));
        cudaMalloc(&d_word_counts, current_chunk_size * sizeof(int));
        cudaMemset(d_word_counts, 0, current_chunk_size * sizeof(int));

        //Se copian los datos del host al GPU, y se lanza el kernel count_words_kernel.
        cudaMemcpy(d_text, h_text, current_chunk_size * sizeof(char), cudaMemcpyHostToDevice);

        int threads_per_block = 1024;
        int blocks = (current_chunk_size + threads_per_block - 1) / threads_per_block;

        count_words_kernel<<<blocks, threads_per_block>>>(d_text, current_chunk_size, d_word_counts);

        cudaDeviceSynchronize();
        checkCudaError(cudaGetLastError(), "Error in count_words_kernel");

        cudaMemcpy(h_word_counts, d_word_counts, current_chunk_size * sizeof(int), cudaMemcpyDeviceToHost);

        // identificando las palabras y contando cuántas veces aparecen. Utiliza el array h_word_counts para encontrar los inicios
        //de las palabras y luego construye cada palabra completa para actualizar su conteo en el mapa total_word_count.
        for (std::size_t i = 0; i < current_chunk_size;) {
            if (h_word_counts[i] > 0) {
                std::string word;
                while (i < current_chunk_size && is_word_char(h_text[i])) {
                    word += h_text[i];
                    ++i;
                }
                total_word_count[word]++;
            } else {
                ++i;
            }
        }

        cudaFree(d_text);
        cudaFree(d_word_counts);
    }

    file.close();

    std::cout << "Total words found: " << total_word_count.size() << std::endl;

    for (const auto &pair : total_word_count) {
        std::cout << pair.first << ": " << pair.second << std::endl;
    }

    delete[] h_text;
    delete[] h_word_counts;

    return 0;
}


Writing wordcounter.cu


In [7]:
!nvcc wordcounter.cu -o wordcounter

In [8]:
!./wordcounter

Se truncaron las últimas líneas 5000 del resultado de transmisión.
liveliness: 46936
decree: 11734
mix: 46936
affair: 70405
style: 316829
pleasantly: 93876
fish: 23469
company: 410705
entire: 93877
unequal: 82139
fancied: 105613
complaisance: 23469
ungracious: 46938
all: 9833380
slight: 199484
eye: 363770
senses: 46939
inconsistent: 11734
quit: 46936
Living: 11735
persuasion: 129077
perplexing: 11734
fashion: 58672
planned: 23469
unfrequently: 11735
labouring: 11735
nicely: 58672
huswife: 23470
comprehended: 23469
shared: 23468
Richardson: 11735
Abbey: 363770
previously: 23469
ardent: 23469
Donwell: 574976
justest: 11735
Fortunately: 23469
Westons: 93875
tempt: 46937
horror: 93875
field: 23470
faces: 35205
stirred: 11734
successless: 11735
gratitude: 340295
busiest: 11734
Any: 35203
outstepped: 11734
rumour: 23470
tens: 11734
hate: 46937
circulate: 11734
pair: 82140
newly: 11735
solitary: 35204
hindered: 11734
egg: 35205
troublesome: 105612
puzzling: 11735
locked: 11734
confirmation: 3